### Course Popularity - Final Project for CSC440: Data Mining


In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import gc

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
from xgboost import XGBRegressor

# ============================================================
# 0. Load CSVs
# ============================================================
sections_file_path = Path("Data") / "Course_data" / "courses.sections.csv"
sectionsTS_file_path = Path("Data") / "Course_data" / "courses.sectionsTS.csv"

sections_df = pd.read_csv(sections_file_path)
sections_ts_df = pd.read_csv(sectionsTS_file_path)

print("\n========== Loaded Datasets ==========")
print("courses.sections.csv info:")
print(sections_df.info(), "\n")

print("courses.sectionsTS.csv info:")
print(sections_ts_df.info(), "\n")

print("Static sections columns:", sections_df.columns)
print("Time-series columns:", sections_ts_df.columns)

FileNotFoundError: [Errno 2] No such file or directory: 'Data/Course_data/courses.sections.csv'

In [2]:
# ============================================================
# 1. Helper: Build Data Quality Report for a DataFrame
# ============================================================

def build_dq_report(df: pd.DataFrame, df_name: str):
    """
    Perform data quality checks and print all results:
    - Overview
    - Null summary (all columns)
    - Dtype summary
    - Numeric summary (all numeric columns)
    - Categorical summary (all categorical columns, full value counts)
    - Type coercion results
    """
    print("\n\n============================================================")
    print(f"############  DATA QUALITY REPORT: {df_name}  ############")
    print("============================================================\n")

    # ---------- Overview ----------
    print("---- Overview ----")
    overview = pd.DataFrame({
        "n_rows": [df.shape[0]],
        "n_columns": [df.shape[1]],
        "n_duplicates": [df.duplicated().sum()]
    })
    print(overview, "\n")

    # ---------- Null Summary (All Columns) ----------
    print("---- Null Summary (All Columns) ----")
    null_summary = pd.DataFrame({
        "column": df.columns,
        "dtype": df.dtypes.values.astype(str),
        "null_count": df.isna().sum().values,
        "null_pct": (df.isna().mean() * 100).values
    }).sort_values("null_pct", ascending=False)

    print(null_summary.to_string(index=False), "\n")

    # ---------- Dtype Summary (All Types) ----------
    print("---- Dtype Summary ----")
    dtype_summary = (
        null_summary[["column", "dtype"]]
        .groupby("dtype")
        .agg(n_columns=("column", "count"),
             columns=("column", lambda x: ", ".join(x)))
        .reset_index()
    )
    print(dtype_summary.to_string(index=False), "\n")

    # ---------- Numeric Summary (All Numeric Columns) ----------
    print("---- Numeric Summary ----")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        numeric_summary = df[numeric_cols].describe().T
        numeric_summary["missing_count"] = df[numeric_cols].isna().sum()
        numeric_summary["missing_pct"] = df[numeric_cols].isna().mean() * 100
        print(numeric_summary.to_string(), "\n")
    else:
        print("No numeric columns.\n")

    # ---------- Categorical Summary (All Categorical Columns) ----------
    print("---- Categorical Summary ----")
    cat_cols = df.select_dtypes(include=["object", "category"]).columns
    cat_summary_rows = []
    for col in cat_cols:
        series = df[col]
        top_values = ", ".join(
            [f"{idx}({val})" for idx, val in series.value_counts(dropna=True).items()]
        )

        cat_summary_rows.append({
            "column": col,
            "dtype": str(series.dtype),
            "n_unique": series.nunique(dropna=True),
            "missing_pct": series.isna().mean() * 100,
            "value_counts": top_values
        })
    if cat_summary_rows:
        cat_summary = pd.DataFrame(cat_summary_rows)
        print(cat_summary.to_string(index=False), "\n")
    else:
        print("No categorical columns.\n")

    # ---------- Type Coercion Checks ----------
    print("---- Type Coercion Checks ----")

    expected_numeric_cols = [
        "capacity", "waitlistCapacity", "reservedSeatsAvailable",
        "reservedSeatsCapacity", "year", "catalogNumber", "classNumber"
    ]
    expected_datetime_cols = [
        "dateTimeRetrieved", "timeStart", "timeEnd",
        "startDate", "endDate"
    ]

    # Numeric Type Coercion
    print("\nNumeric Columns Coercion Results:")
    for col in expected_numeric_cols:
        if col in df.columns:
            before = df[col].notna().sum()
            coerced = pd.to_numeric(df[col], errors="coerce")
            after = coerced.notna().sum()
            print(
                f"{col}: non-null before={before}, after={after}, "
                f"invalid converted={before - after}"
            )
            df[col] = coerced

    # Datetime Type Coercion
    print("\nDatetime Columns Coercion Results:")
    for col in expected_datetime_cols:
        if col in df.columns:
            before = df[col].notna().sum()
            coerced = pd.to_datetime(df[col], errors="coerce")
            after = coerced.notna().sum()
            print(
                f"{col}: non-null before={before}, after={after}, "
                f"invalid converted={before - after}"
            )
            df[col] = coerced

    print("\n========== END OF REPORT FOR:", df_name, "==========\n")

    return df


# ============================================================
# 2. Run Reports for Both Datasets
# ============================================================

sections_df = build_dq_report(sections_df.copy(), "courses.sections")
sections_ts_df = build_dq_report(sections_ts_df.copy(), "courses.sectionsTS")

gc.collect()

print("\nData quality analysis completed.\n")


NameError: name 'sections_df' is not defined

In [3]:
# ============================================================
# Fix duplicate / typo columns: waitlistAvailable vs wailistAvailable
# ============================================================
if "waitlistAvailable" in sections_ts_df.columns and "wailistAvailable" in sections_ts_df.columns:
    col_good = sections_ts_df["waitlistAvailable"]
    col_typo = sections_ts_df["wailistAvailable"]

    # Where both have values and differ, count mismatches
    both_non_null = col_good.notna() & col_typo.notna()
    mismatches = (both_non_null & (col_good != col_typo)).sum()

    print(f"\n[Data Fix] waitlistAvailable vs wailistAvailable:")
    print(f"  Rows where BOTH are non-null: {both_non_null.sum()}")
    print(f"  Rows where values MISMATCH:  {mismatches}")

    # Merge logic (intersection-style):
    # - If both non-null and equal → keep that value
    # - If only one is non-null       → use the non-null value
    # - If both non-null and different → set NaN (and rely on later checks)
    merged = np.where(
        both_non_null & (col_good == col_typo),
        col_good,                     # agree → keep
        np.where(
            col_good.notna() & ~col_typo.notna(),
            col_good,                 # only good has value
            np.where(
                col_typo.notna() & ~col_good.notna(),
                col_typo,             # only typo has value
                np.nan                # mismatch or both null
            )
        )
    )

    sections_ts_df["waitlistAvailable"] = merged

    # Drop the bad column
    sections_ts_df = sections_ts_df.drop(columns=["wailistAvailable"])

    print("  -> Merged into 'waitlistAvailable' and dropped 'wailistAvailable'.\n")

else:
    print("\n[Data Fix] One or both columns 'waitlistAvailable' / 'wailistAvailable' not found; no merge performed.\n")


# ===============================================================
# 2. Merge Static Course Metadata with Time-Series Observations
# ===============================================================

# Merge on courseInfo.classNumber (primary key)
df = sections_ts_df.merge(
    sections_df,
    on="courseInfo.classNumber",
    how="left",
    suffixes=("", "_static")
)

# Convert timestamps
df["dateTimeRetrieved"] = pd.to_datetime(df["dateTimeRetrieved"], errors="coerce")
df = df.sort_values(["courseInfo.classNumber", "dateTimeRetrieved"])

# ===============================================================
# 3. Feature Engineering
# ===============================================================

# ---- Temporal Features ----
df["days_since_start"] = df.groupby("courseInfo.classNumber")["dateTimeRetrieved"] \
                           .transform(lambda x: (x - x.min()).dt.total_seconds() / 86400)

df["hour"] = df["dateTimeRetrieved"].dt.hour
df["day_of_week"] = df["dateTimeRetrieved"].dt.dayofweek # 0=Monday, ..., 6=Sunday
df["is_weekend"] = df["day_of_week"].isin([5,6]).astype(int) # 5=Saturday, 6=Sunday

# Cyclical time encoding
df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)
df["dow_sin"] = np.sin(2 * np.pi * df["day_of_week"] / 7)
df["dow_cos"] = np.cos(2 * np.pi * df["day_of_week"] / 7)

# ---- Enrollment Features ----
df["seats_taken"] = df["capacity"] - df["seatsAvailable"]
df["fill_pct"] = df["seats_taken"] / df["capacity"].replace(0, np.nan)

df["waitlist_ratio"] = df["waitlistAvailable"] / df["waitlistCapacity"].replace(0, np.nan)

# ---- Velocity Metrics ----
df["fill_velocity_per_day"] = df.groupby("courseInfo.classNumber")["fill_pct"].diff() / \
                              df.groupby("courseInfo.classNumber")["days_since_start"].diff()

df["rolling_velocity_6h"] = df.groupby("courseInfo.classNumber")["fill_pct"].transform(
    lambda x: x.diff().rolling(6).mean()
)

# ---- Course-Level Features ----
df["course_level"] = df["catalogNumber"] // 100
df["is_online"] = (df["classroom"] == "Online").astype(int)

df["timeStart"] = pd.to_datetime(df["timeStart"], errors="coerce")
df["early_morning"] = (df["timeStart"].dt.hour < 10).astype(int)
df["evening"] = (df["timeStart"].dt.hour >= 17).astype(int)

gc.collect()


# ===============================================================
# 4. Modeling Dataset Construction
# ===============================================================
target = "fill_pct"

feature_cols = [
    "days_since_start", "hour_sin", "hour_cos", "dow_sin", "dow_cos",
    "is_weekend", "seats_taken", "capacity", "waitlist_ratio",
    "fill_velocity_per_day", "rolling_velocity_6h",
    "course_level", "is_online", "early_morning", "evening"
]

# One-hot encode subjectCode & academicCareer
df = pd.get_dummies(df, columns=["subjectCode", "academicCareer"], drop_first=True)

feature_cols.extend([col for col in df.columns if col.startswith("subjectCode_")])
feature_cols.extend([col for col in df.columns if col.startswith("academicCareer_")])

df = df.dropna(subset=[target])

X = df[feature_cols]
y = df[target]

# ===============================================================
# 5. Time-Series Cross Validation (No Leakage)
# ===============================================================
tscv = TimeSeriesSplit(n_splits=5)

mae_scores, rmse_scores, r2_scores = [], [], []

model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    tree_method="hist"
)

for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    mae_scores.append(mean_absolute_error(y_test, preds))
    rmse_scores.append(mean_squared_error(y_test, preds, squared=False))
    r2_scores.append(r2_score(y_test, preds))


print("\n========== Time-Series CV Results ==========")
print("MAE:", np.mean(mae_scores))
print("RMSE:", np.mean(rmse_scores))
print("R²:", np.mean(r2_scores))


# ===============================================================
# 6. Business-Relevant Metric: ±5% Tolerance Accuracy
# ===============================================================
model.fit(X, y)
preds = model.predict(X)

tolerance = 0.05
within_tol = np.mean(np.abs(preds - y) <= tolerance)

print("\nWithin ±5% tolerance accuracy:", within_tol)


# ===============================================================
# 7. Popularity Metrics for Ranking Courses
# ===============================================================
course_groups = df.groupby("courseInfo.classNumber")

popularity = pd.DataFrame({
    "courseInfo.classNumber": course_groups["courseInfo.classNumber"].first(),
    "subjectCode": course_groups["subjectCode"].first(),
    "catalogNumber": course_groups["catalogNumber"].first(),
    "course_level": course_groups["course_level"].first(),
    
    # Popularity Scores:
    "final_fill_rate": course_groups["fill_pct"].last(),
    "fill_velocity_score": course_groups["fill_velocity_per_day"].mean(),
    "early_rush_score": course_groups["fill_pct"].nth(3),  # first few hours
    "waitlist_demand_score": course_groups["waitlist_ratio"].max()
})

# Composite Score
popularity["popularity_score"] = (
    0.40 * popularity["final_fill_rate"] +
    0.30 * popularity["fill_velocity_score"] +
    0.20 * popularity["waitlist_demand_score"] +
    0.10 * popularity["early_rush_score"]
)

# ===============================================================
# 8. Ranking Courses (CSC, ECE by 200/300/400 level)
# ===============================================================
target_subjects = ["CSC", "ECE"]
filtered = popularity[popularity["subjectCode"].isin(target_subjects)]

ranked_output = {}

for subj in target_subjects:
    subj_df = filtered[filtered["subjectCode"] == subj]
    ranked_output[subj] = {}

    for level in [200, 300, 400]:
        lvl = subj_df[subj_df["course_level"] == level]
        ranked_output[subj][level] = lvl.sort_values(
            "popularity_score", ascending=False
        ).head(10)  # top 10

# Print Sample
print("\n=========== Top CSC 200-Level Courses ===========")
print(ranked_output["CSC"][200].head())

print("\n=========== Top ECE 300-Level Courses ===========")
print(ranked_output["ECE"][300].head())

# Optionally export results
popularity.to_csv("course_popularity_scores.csv", index=False)


NameError: name 'sections_ts_df' is not defined